# Main Feedback Generation Notebook
This notebook handles preprocessing, model interaction, and feedback generation.

**--Set up: Github, Paths, Imports**

In [1]:
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
# !git branch -a

/content
fatal: destination path 'project' already exists and is not an empty directory.
/content/project


In [2]:
# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout -b model/prompting origin/model/prompting

fatal: A branch named 'model/prompting' already exists.


**--Install required libraries**

In [3]:
%pip install -r requirements.txt --quiet

In [4]:
# Add src folder to python path to edit python files
import sys
sys.path.append('/content/project/')

from google.colab import userdata
import os
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key

**--Import modules (youre working on)**
<br>*each of our classes will be what will merge to this notebook (im p sure)*
<br>only loads what you explicitly request
<br>(this helps keep memory low and import fast)
<br> *the following is an example with my Rubric module*

In [5]:
!pip install --upgrade openai --quiet

In [6]:
from tropos import StudentSubmission
from tropos.models import gpt
from tropos.docx_writer import write_feedback_to_docx
from utils.feedback_formatting import format_feedback_blocks
import os

#load first submission with cached requirements
requirements_path = "/content/project/data/raw/Requirements.docx"
exampleStudent1part1 = StudentSubmission("/content/project/data/raw/Student_1/Student_1_Part_1.docx", requirements_path)

#----one student across multiple parts
exampleStudent1part2 = StudentSubmission("/content/project/data/raw/Student_1/Student_1_Part_2.docx", requirements_path)
exampleStudent1part3 = StudentSubmission("/content/project/data/raw/Student_1/Student_1_Part_3.docx", requirements_path)
exampleStudent1partfinal = StudentSubmission("/content/project/data/raw/Student_1/Student_1_Final.docx", requirements_path)

#----diff students part1
exampleStudent2part1 = StudentSubmission("/content/project/data/raw/Student_2/Student_2_Part_1.docx", requirements_path)
exampleStudent3part1 = StudentSubmission("/content/project/data/raw/Student_3/Student_3_Part_1.docx", requirements_path)
exampleStudent4part1 = StudentSubmission("/content/project/data/raw/Student_4/Student_4_Part_1.docx", requirements_path)
exampleStudent5part1 = StudentSubmission("/content/project/data/raw/Student_5/Student_5_Part_1.docx", requirements_path)

#--target papers (uncommented/raw)
target1 = StudentSubmission("/content/project/data/unmarked_raw/Uncommented_Student_1.docx", requirements_path)
target2 = StudentSubmission("/content/project/data/unmarked_raw/Uncommented_Student_2.docx", requirements_path)
target3 = StudentSubmission("/content/project/data/unmarked_raw/Uncommented_Student_3.docx", requirements_path)
target4 = StudentSubmission("/content/project/data/unmarked_raw/Uncommented_Student_4.docx", requirements_path)


#promot example sets
oneshot_example = exampleStudent1part1
fewshot_same_student = [
    exampleStudent1part1,
    exampleStudent1part2,
    exampleStudent1part3,
    exampleStudent1partfinal
]
fewshot_diff_part1 = [
    exampleStudent1part1,
    exampleStudent2part1,
    exampleStudent3part1,
    exampleStudent4part1,
    #exampleStudent5part1
]

#list of targets
targets = [
    ("Student1", target1),
    ("Student2", target2),
    ("Student3", target3),
    ("Student4", target4)
]

Document contains no comments
No tables in document.
Document contains no comments
No tables in document.
Document contains no comments
No tables in document.
Document contains no comments
No tables in document.
Document contains no comments


In [7]:
#prompt builder functions
from tropos.models.gpt import (
    build_zeroshot_prompt,
    build_oneshot_prompt,
    build_fewshot_prompt
)

#define prompt variants
prompt_variants = [
    ("ZeroShot", build_zeroshot_prompt),
    ("OneShot", lambda t: build_oneshot_prompt(
        oneshot_example, t)),
    ("FewShotSameStudent", lambda t: build_fewshot_prompt(
        fewshot_same_student, t)),
    ("FewShotDiffStudents", lambda t: build_fewshot_prompt(
        fewshot_diff_part1, t)),
]


#run all combos
for target_label, target in targets:
    for variant_label, builder in prompt_variants:
        print(f"\n🧠 Generating {variant_label} feedback for {target_label}...")
        prompt = builder(target)
        feedback = gpt.call_chatgpt(prompt)

        print(f"\n--- {variant_label} Feedback for {target_label} ---\n")
        format_feedback_blocks(feedback, width=80)

        output_path = f"/content/project/data/generated_output/{variant_label}_{target_label}.docx"
        write_feedback_to_docx(
            submission_path=target.submission_path,
            feedback_text=feedback,
            output_path=output_path
        )

        print(f"✅ Saved to {output_path}")





🧠 Generating ZeroShot feedback for Student1...

--- ZeroShot Feedback for Student1 ---

📝 Instructor Feedback

🗨️  "Unfortunately, the year 2019 is known as the year of quarantine." 👉 – It's
more accurate to say 2020 was the year of quarantine, as COVID-19 wasn't widely
recognized or impacting daily life until early 2020.

🗨️  "Additionally, the year 2020 is known for the start of mask mandates." 👉 –
This is a good start, but consider expanding on why mask mandates were deemed
necessary.

🗨️  "At first, the public was confused about the new virus." 👉 – This is a broad
statement; consider providing specific examples or references to support this
claim.

🗨️  "The mask mandate refers to the authoritative governmental policy of wearing
a mask in certain settings when out in public." 👉 – This is a clear definition,
but could be enhanced by mentioning specific settings where mandates were
applied.

🗨️  "In this paper, it is important to know the difference between a mandate and
recommendati

In [8]:
import shutil
import os
shutil.make_archive('/content/folder_name', 'zip', '/content/project/data/generated_output')

from google.colab import files
files.download('/content/folder_name.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>